### Ensemble

In [1]:
import pandas as pd
from sklearn.ensemble import VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
cancer = load_breast_cancer()
cancer_df = pd.DataFrame(data = cancer.data, columns = cancer.feature_names)

In [3]:
cancer_df.head(3)

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.8,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.6,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.9,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.8,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.0,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.5,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758


In [4]:
# 개별 모델은 Logistic Regression, KNN 임
lr_clf = LogisticRegression()
knn_clf = KNeighborsClassifier(n_neighbors=8)
# Hard Voting = 단순하게 가장 많은 표를 얻은 결과 선택
# Soft Voting = 모델별로 예측한 Probability 합산
# Weighted Voting = 모델별로 가중치 계산
vo_clf = VotingClassifier(estimators=[('LR',lr_clf),('KNN',knn_clf)], voting='soft')
X_train,X_test,y_train,y_test = train_test_split(cancer_df,cancer.target,test_size=0.2)

In [5]:
import numpy as np
vo_clf.fit(X_train,y_train)
pred = vo_clf.predict(X_test)
print(np.round(accuracy_score(y_test,pred),4))

0.9298


C:\Users\sw993\anaconda3\envs\test1\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [6]:
classifiers = [lr_clf,knn_clf]
for classifier in classifiers:
    classifier.fit(X_train,y_train)
    pred = classifier.predict(X_test)
    accuracy = accuracy_score(y_test,pred)
    class_name = classifier.__class__.__name__
    print('{0}, {1:.4f}'.format(class_name,accuracy))

LogisticRegression, 0.9035
KNeighborsClassifier, 0.9211


C:\Users\sw993\anaconda3\envs\test1\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### Human Activity - Random Forest Classifier 

In [7]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [8]:
feature = pd.read_csv('C:/Users/sw993/PerfectGuide-master/4장/human_activity/features.txt',
                      sep='\s+')
feature_df=feature.iloc[:,1]

In [9]:
new_features = []
for i, n in enumerate(feature_df):
    new_feature = n + '_' + str(i)
    new_features.append(new_feature)

In [19]:
X_train = pd.read_csv('C:/Users/sw993/PerfectGuide-master/4장/human_activity/train/X_train.txt',
                    names = new_features,sep='\s+')
y_train = pd.read_csv('C:/Users/sw993/PerfectGuide-master/4장/human_activity/train/y_train.txt',
                     names = ['target'])
X_test = pd.read_csv('C:/Users/sw993/PerfectGuide-master/4장/human_activity/test/X_test.txt',
                     names = new_features,sep='\s+')
y_test = pd.read_csv('C:/Users/sw993/PerfectGuide-master/4장/human_activity/test/y_test.txt',
                     names = ['target'])

In [23]:
from sklearn.model_selection import GridSearchCV
rf_clf = RandomForestClassifier()
params = {
    'n_estimators':[100],
    'max_depth' : [6, 8, 10, 12], 
    'min_samples_leaf' : [8, 12, 18 ],
    'min_samples_split' : [8, 16, 20]
}
grid_cv = GridSearchCV(rf_clf, param_grid = params, cv=3,scoring='accuracy')
grid_cv.fit(X_train,y_train)

GridSearchCV(cv=3, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [6, 8, 10, 12],
                         'min_samples_leaf': [8, 12, 18],
                         'min_samples_split': [8, 16, 20],
                         'n_estimators': [100]},
             scoring='accuracy')

In [24]:
print(grid_cv.best_score_)
print(grid_cv.best_params_)

0.9213823040436084
{'max_depth': 12, 'min_samples_leaf': 12, 'min_samples_split': 16, 'n_estimators': 100}


In [34]:
best_clf = grid_cv.best_estimator_
feature_importance = best_clf.feature_importances_

importance = pd.Series(feature_importance,index=X_train.columns).sort_values(ascending=False)[:20]

In [36]:
importance.head(3)

tGravityAcc-mean()-X_39    0.040454
tGravityAcc-min()-X_51     0.034254
tGravityAcc-max()-Y_49     0.028020
dtype: float64

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(figsize=(8,8))
plt.title('human_activity feature_importance')
sns.barplot(x=importance, y = importance.index)
plt.show

### GBM
###### 수행속도가 매우 느리기 때문에, LightGBM이나 XGBoost 이용

In [39]:
from sklearn.ensemble import GradientBoostingClassifier
import time
import warnings
warnings.filterwarnings('ignore')

start_time = time.time()
gb_clf = GradientBoostingClassifier()
gb_clf.fit(X_train,y_train)
gb_pred = gb_clf.predict(X_test)
gb_accuracy = accuracy_score(y_test,gb_pred)
print(time.time()-start_time,np.round(gb_accuracy,4))

696.1705632209778 0.9393


In [ ]:
gb_clf = GradientBoostingClassifier()
params = {
    'n_estimators':[100, 500],
    'learning_rate' : [ 0.05, 0.1]
}
grid_cv = GridSearchCV(gb_clf,param_grid=params,cv=3,verbose=0,scoring='accuracy')
start_time = time.time()
grid_cv.fit(X_train,y_train)
print(time.time()-start_time)

In [ ]:
print(grid_cv.cv_results_)

In [ ]:
print(grid_cv.best_score_)
print(grid_cv.best_params_)

In [ ]:
best_estimator = grid_cv.best_estimator_
best_estimator.fit(X_train,y_train)
pred = best_estimator.predict(X_test)
accuracy = accuracy_score(y_test,pred)
print('{0:.4f}'.fotmat(accuracy))